## 准备工作

生成稀疏矩阵

In [125]:
import numpy as np
import random
sparse=0.35
size=4 #生成方阵的大小：size*size
matrix=np.arange(size*size).reshape(size,size)
for i in range(len(matrix)):
    for j in range(len(matrix[i])):
        x=random.randint(0,100)
        if x > (100*sparse):
            matrix[i][j]=0
        else:
            matrix[i][j]=x+1
print("Generate random matirx")
print(matrix)

Generate random matirx
[[ 0  0  0  0]
 [ 0  0  0 10]
 [ 0  0  8  0]
 [ 0  0  0  0]]


普通矩阵转CSR矩阵函数

In [126]:
# matrix to csr format
def matrix2csr(matrix):
    row_num=0
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] != 0:
                row_num+=1
    values=[1]*row_num
    columnIndex=[1]*row_num
    rowPtr=[1]*(len(matrix)+1)
    k=0
    l=0
    tmp_size=list(range(len(matrix)))
    for i in tmp_size:
        for j in tmp_size:
            if j==0:
                rowPtr[l]=k
                l+=1
            if matrix[i][j]!=0:
                values[k]=matrix[i][j]
                columnIndex[k]=j
                k+=1
    rowPtr[l]=row_num
    return values,columnIndex,rowPtr

将普通稀疏矩阵转为CSR型稀疏矩阵

In [127]:
values=[]
rowPtr=[]
columnIndex=[]
values,columnIndex,rowPtr=matrix2csr(matrix)

生成随机向量

In [128]:
x=[1]*size
for i in list(range(len(x))):
    x[i]=random.randint(0,100)-20
x=np.float32(x)
values=np.float32(values)
columnIndex=np.int32(columnIndex)
rowPtr=np.int32(rowPtr)

In [129]:
print("随机向量：")
print(x)
print("数据数组：")
print(values)
print("列索引：")
print(columnIndex)
print("行索引：")
print(rowPtr)

随机向量：
[73. 69. 75. 50.]
数据数组：
[10.  8.]
列索引：
[3 2]
行索引：
[0 0 1 2 2]


## 加载Overlay模块

In [130]:
from pynq import Overlay
overlay = Overlay("./CRS2.bit")
CRS = overlay.spmv_wrap_0

## 分配内存供IP使用

In [131]:
from pynq import allocate
values_buffer = allocate(shape=(len(values)), dtype=np.float32)
columnIndex_buffer = allocate(shape=(len(columnIndex)), dtype=np.int32)
rowPtr_buffer = allocate(shape=(len(rowPtr)), dtype=np.int32)
output_buffer = allocate(shape=(4), dtype=np.float32)
x_buffer = allocate(shape=(len(x)), dtype=np.float32)

In [132]:
np.copyto(values_buffer, np.float32(values))
np.copyto(columnIndex_buffer, columnIndex)
np.copyto(rowPtr_buffer, rowPtr)
np.copyto(x_buffer, np.float32(x))

In [133]:
print("随机向量：")
print(x_buffer)
print("数据数组：")
print(values_buffer)
print("列索引：")
print(columnIndex_buffer)
print("行索引：")
print(rowPtr_buffer)

随机向量：
[73. 69. 75. 50.]
数据数组：
[10.  8.]
列索引：
[3 2]
行索引：
[0 0 1 2 2]


In [134]:
CRS.s_axi_control.write(0x1c,columnIndex_buffer.physical_address)
CRS.s_axi_control.write(0x10,rowPtr_buffer.physical_address)
CRS.s_axi_control.write(0x28,values_buffer.physical_address)
CRS.s_axi_control.write(0x40,x_buffer.physical_address)
CRS.s_axi_control.write(0x34,output_buffer.physical_address)

## 启动IP

In [135]:
import time
CRS.s_axi_CTRL.write(0x00, 0x01)
while True:
    reg = CRS.s_axi_CTRL.read(0x00)
    if reg != 1:
        break

## 计算结果

IP计算结果

In [136]:
print(output_buffer)

[  0. 500. 600.   0.]


正确计算答案

In [137]:
result=np.dot(matrix, x)

In [138]:
print(result)

[  0. 500. 600.   0.]


## 判断计算结果True或False

In [124]:
print((result==output_buffer).all() )

True
